In [1]:
from Bio.PDB import PDBParser, PDBIO
from Bio.PDB.Structure import Structure as BStructure
from Bio.PDB.Model import Model as BModel
from Bio.PDB.Chain import Chain as BChain
from Bio.PDB.Residue import Residue as BResidue
from Bio.PDB.Atom import Atom as BAtom

In [36]:
import json
import math
import glob
import pickle
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

from utils import *

# data

In [3]:
path = "./data/cdr_seqs_20221128.json"

with open(path, "rb") as f:
    cdr_seqs = json.loads(f.read())

print(len(cdr_seqs.keys()))
cdr_seqs

6030


{'2hh0': {'HL': {'L1': 'QDIGNN',
   'L2': 'ATS',
   'L3': 'LQHDTFPLT',
   'H1': 'GFNIEDSY',
   'H2': 'IDPEDGET',
   'H3': 'GRGAYYIKEDF'}},
 '1mhp': {'XY': {'L1': 'SSVNH',
   'L2': 'LTS',
   'L3': 'QQWSGNPWT',
   'H1': 'GFTFSRYT',
   'H2': 'ISGGGHT',
   'H3': 'TRGFGDGGYFDV'},
  'HL': {'L1': 'SSVNH',
   'L2': 'LTS',
   'L3': 'QQWSGNPWT',
   'H1': 'GFTFSRYT',
   'H2': 'ISGGGHT',
   'H3': 'TRGFGDGGYFDV'}},
 '1mhh': {'DC': {'L1': 'QSLLNSRTRKNY',
   'L2': 'WAS',
   'L3': 'KQAYIPPLT',
   'H1': 'GYTFTDFS',
   'H2': 'VNTETGEP',
   'H3': 'ARFLLRQYFDV'},
  'BA': {'L1': 'QSLLNSRTRKNY',
   'L2': 'WAS',
   'L3': 'KQAYIPPLT',
   'H1': 'GYTFTDFS',
   'H2': 'VNTETGEP',
   'H3': 'ARFLLRQYFDV'}},
 '7st3': {'Z': {'H1': 'GSIFSINT', 'H2': 'ISSGGST', 'H3': 'YGLSYSNDDY'},
  'd': {'H1': 'GSIFSINT', 'H2': 'ISSGGST', 'H3': 'YGLSYSNDDY'},
  'N': {'H1': 'GSIFSINT', 'H2': 'ISSGGST', 'H3': 'YGLSYSNDDY'},
  'X': {'H1': 'GSIFSINT', 'H2': 'ISSGGST', 'H3': 'YGLSYSNDDY'},
  'b': {'H1': 'GSIFSINT', 'H2': 'ISSGGST', 'H3': 

In [4]:
cdr_seqs["7sk7"]

{'DC': {'L1': 'QSVSSA',
  'L2': 'SAS',
  'L3': 'QQYYYPLFT',
  'H1': 'GFNFSYSS',
  'H2': 'IYSSYGYT',
  'H3': 'ARVYPWWYYKYYHGALDY'},
 'K': {'H1': 'GRTISRYA', 'H2': 'ARRSGDGA', 'H3': 'AIDSDTFYSGSYDY'}}

In [5]:
cdr_seqs["7t6s"]

{'EE': {'L1': 'KSLLHSNGNTY',
  'L2': 'RMS',
  'L3': 'MQHLEYPLT',
  'H1': 'GFAFSSFG',
  'H2': 'ISSGSGTI',
  'H3': 'VRSIYYYGSSPFDF'}}

In [6]:
summary_file = pd.read_csv("../../MSAI_Project/SAbDab_20221124/sabdab_summary_all.tsv", sep="\t")
summary_file

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,compound,organism,heavy_species,light_species,antigen_species,authors,resolution,method,r_free,r_factor,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid
0,7t17,J,K,0,C,protein,NaN,core protein,VIRUS,11/23/22,Zika Virus asymmetric unit bound with IgM anti...,Homo sapiens; Zika virus,homo sapiens,homo sapiens,zika virus,"Miller, A.S., Kuhn, R.J.",0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV4,IGLV1,Lambda,None,None,None,None,None
1,7t17,F,G,0,A,protein,NaN,core protein,VIRUS,11/23/22,Zika Virus asymmetric unit bound with IgM anti...,Homo sapiens; Zika virus,homo sapiens,homo sapiens,zika virus,"Miller, A.S., Kuhn, R.J.",0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV4,IGLV1,Lambda,None,None,None,None,None
2,6fe4,F,NaN,0,A,protein,NaN,shiga-like toxin 2 subunit b,TOXIN,12/29/17,Crystal structure of the complex between Shiga...,ENTEROBACTERIA PHAGE 933W; VICUGNA PACOS,vicugna pacos,NaN,enterobacteria phage 933w,"Bernedo, R., Muyldermans, S., Sterckx, Y.G.J.",3,X-RAY DIFFRACTION,0.207,0.169,False,True,IGHV3,NaN,NaN,9.60E-09,-10.93813908,SPR,NaN,TBD
3,7jmo,H,L,0,A,protein,NaN,spike protein s1,IMMUNE SYSTEM,2008/2/20,Crystal structure of SARS-CoV-2 receptor bindi...,SEVERE ACUTE RESPIRATORY SYNDROME CORONAVIRUS ...,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus2,"Wu, N.C., Yuan, M., Liu, H., Zhu, X., Wilson, ...",2.359,X-RAY DIFFRACTION,0.238,0.195,False,True,IGHV3,IGKV3,Kappa,None,None,None,None,None
4,7sgf,H,L,0,A | a,protein | protein,NA | NA,gpc-i53-50a | unknown,VIRAL PROTEIN/IMMUNE SYSTEM,2010/12/22,Lassa virus glycoprotein construct (Josiah GPC...,Lassa mammarenavirus; Oryctolagus cuniculus,oryctolagus cuniculus,oryctolagus cuniculus,lassa mammarenavirus | lassa mammarenavirus,"Antanasijevic, A., Brouwer, P.J.M., Ward, A.B.",4.41,ELECTRON MICROSCOPY,NaN,NaN,False,True,unknown,unknown,unknown,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13426,7lo6,J,I,0,C,protein,NaN,envelope glycoprotein bg505 sosip.664 gp120,VIRAL PROTEIN/IMMUNE SYSTEM,04/14/21,Structure of CD4 mimetic BNM-III-170 in comple...,HUMAN IMMUNODEFICIENCY VIRUS 1; HOMO SAPIENS,homo sapiens,homo sapiens,human immunodeficiency virus 1,"Jette, C.A., Bjorkman, P.J.",3.9,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV1,IGKV3,Kappa,None,None,None,None,None
13427,3vi3,H,L,0,D,protein,NaN,integrin beta-1,CELL ADHESION/IMMUNE SYSTEM,09/21/11,Crystal structure of alpha5beta1 integrin head...,HOMO SAPIENS; MUS MUSCULUS,mus musculus,mus musculus,homo sapiens,"Nagae, M., Nogi, T., Takagi, J.",2.9,X-RAY DIFFRACTION,0.267,0.207,False,True,IGHV1,IGKV2,Kappa,None,None,None,None,None
13428,6zdg,F,G,0,D,protein,NaN,spike glycoprotein,VIRAL PROTEIN,06/14/20,Association of three complexes of largely stru...,SEVERE ACUTE RESPIRATORY SYNDROME CORONAVIRUS ...,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus2,"Duyvesteyn, H.M.E., Zhou, D., Zhao, Y., Fry, E...",4.7,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV3,IGKV1,Kappa,None,None,None,None,None
13429,7sk7,K,NaN,0,NaN,NaN,NaN,NaN,SIGNALING PROTEIN/IMMUNE SYSTEM,07/27/22,Cryo-EM structure of human ACKR3 in complex wi...,Homo sapiens; Lama glama,lama glama,NaN,NaN,"Yen, Y.C., Schafer, C.T., Gustavsson, M., Hand...",0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV1,NaN,NaN,None,None,None,None,None


In [7]:
summary_file[summary_file["pdb"]=="7sk7"]

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,compound,organism,heavy_species,light_species,antigen_species,authors,resolution,method,r_free,r_factor,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid
4898,7sk7,D,C,0,B | A,protein | protein,NA | NA,stromal cell-derived factor 1 | atypical chemo...,SIGNALING PROTEIN/IMMUNE SYSTEM,07/27/22,Cryo-EM structure of human ACKR3 in complex wi...,Homo sapiens; Lama glama,homo sapiens,homo sapiens,homo sapiens | homo sapiens,"Yen, Y.C., Schafer, C.T., Gustavsson, M., Hand...",0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV3,IGKV1,Kappa,None,None,None,None,None
13429,7sk7,K,NaN,0,NaN,NaN,NaN,NaN,SIGNALING PROTEIN/IMMUNE SYSTEM,07/27/22,Cryo-EM structure of human ACKR3 in complex wi...,Homo sapiens; Lama glama,lama glama,NaN,NaN,"Yen, Y.C., Schafer, C.T., Gustavsson, M., Hand...",0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV1,NaN,NaN,None,None,None,None,None


### filtering protocol
- resolution better than 3A
- have VH and VL
- any antibody sequences <95% similarity
- at least 5 residues in contact with antigen

E. Liberis, P. Veličković, P. Sormanni, M. Vendruscolo, and P. Liò, "Parapred: antibody paratope prediction using convolutional and recurrent neural networks," Bioinformatics, vol. 34, no. 17, pp. 2944-2950, 2018.


### my protocol:
- resolution <4A
- have VH, VL, VA

In [8]:
def filtering(row):
    Hchain = row["Hchain"]
    Lchain = row["Lchain"]
    Achain = row["antigen_chain"]
    res = row["resolution"]

    # and float(str(res).split()[-1])>0 # Q: what does it mean when resolution=0?
    if str(Hchain)!="nan" and \
        str(Lchain)!="nan" and \
        str(Achain)!="nan" and \
        str(res)!="NOT" and \
        float(str(res).split()[-1])<4.0 and \
        str(Hchain)!=str(Lchain) and \
        str(Hchain) not in str(Achain).split(" | ") and \
        str(Lchain) not in str(Achain).split(" | "):
        
#         str(Hchain).upper() not in str(Achain).upper().split(" | ") and \
#         str(Lchain).upper() not in str(Achain).upper().split(" | "):
        
        return 1
    else:
        return 0

summary_file["valid"] = summary_file.apply(filtering, axis=1)
summary_file = summary_file[summary_file["valid"]==1]
summary_file

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,compound,organism,heavy_species,light_species,antigen_species,authors,resolution,method,r_free,r_factor,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid,valid
0,7t17,J,K,0,C,protein,NaN,core protein,VIRUS,11/23/22,Zika Virus asymmetric unit bound with IgM anti...,Homo sapiens; Zika virus,homo sapiens,homo sapiens,zika virus,"Miller, A.S., Kuhn, R.J.",0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV4,IGLV1,Lambda,None,None,None,None,None,1
1,7t17,F,G,0,A,protein,NaN,core protein,VIRUS,11/23/22,Zika Virus asymmetric unit bound with IgM anti...,Homo sapiens; Zika virus,homo sapiens,homo sapiens,zika virus,"Miller, A.S., Kuhn, R.J.",0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV4,IGLV1,Lambda,None,None,None,None,None,1
3,7jmo,H,L,0,A,protein,NaN,spike protein s1,IMMUNE SYSTEM,2008/2/20,Crystal structure of SARS-CoV-2 receptor bindi...,SEVERE ACUTE RESPIRATORY SYNDROME CORONAVIRUS ...,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus2,"Wu, N.C., Yuan, M., Liu, H., Zhu, X., Wilson, ...",2.359,X-RAY DIFFRACTION,0.238,0.195,False,True,IGHV3,IGKV3,Kappa,None,None,None,None,None,1
6,4o51,B,A,0,N,peptide,NaN,ides hinge peptide,IMMUNE SYSTEM,12/19/13,Crystal structure of the QAA variant of anti-h...,"ORYCTOLAGUS CUNICULUS, HOMO SAPIENS; SYNTHETIC...","oryctolagus cuniculus, homo sapiens","oryctolagus cuniculus, homo sapiens",homo sapiens,"Malia, T.J., Luo, J., Teplyakov, A., Gilliland...",2.204,X-RAY DIFFRACTION,0.213,0.182,False,True,IGHV1,IGKV1,Kappa,None,None,None,None,None,1
9,7orb,E,F,0,X,protein,NaN,spike protein s1,VIRAL PROTEIN,2007/7/21,Crystal structure of the L452R mutant receptor...,SEVERE ACUTE RESPIRATORY SYNDROME CORONAVIRUS ...,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus2,"Zhou, D., Ren, J., Stuart, D.I.",2.5,X-RAY DIFFRACTION,0.251,0.213,False,True,IGHV3,IGKV1,Kappa,None,None,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13423,7tas,H,L,0,E,protein,NaN,spike glycoprotein,VIRAL PROTEIN/IMMUNE SYSTEM,2001/12/22,SARS-CoV-2 spike in complex with the S2K146 ne...,HOMO SAPIENS; SEVERE ACUTE RESPIRATORY SYNDROM...,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus2,"Park, Y.J., Veesler, D., Seattle Structural Ge...",3.2,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV3,IGLV1,Lambda,None,None,None,None,None,1
13424,7wk0,A,B,0,C,protein,NaN,spike protein s1,VIRAL PROTEIN,07/13/22,Local refine of Omicron spike bitrimer with 6m...,Homo sapiens; Severe acute respiratory syndrom...,homo sapiens,homo sapiens,severe acute respiratory syndrome coronavirus2,"Zhan, W.Q., Zhang, X., Chen, Z.G., Sun, L.",0,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV3,IGLV3,Lambda,None,None,None,None,None,1
13425,6ejm,H,h,0,B,protein,NaN,cd81 antigen,CELL ADHESION,09/22/17,CRYSTAL STRUCTURE OF HUMAN CD81 LARGE EXTRACEL...,HOMO SAPIENS; MUS MUSCULUS,mus musculus,NaN,homo sapiens,"Kuglstatter, A., Harris, S.F., Villasenor, A.",2.15,X-RAY DIFFRACTION,0.278,0.215,True,True,unknown,unknown,unknown,8.60E-10,-12.36755691,SPR,NaN,TBD,1
13426,7lo6,J,I,0,C,protein,NaN,envelope glycoprotein bg505 sosip.664 gp120,VIRAL PROTEIN/IMMUNE SYSTEM,04/14/21,Structure of CD4 mimetic BNM-III-170 in comple...,HUMAN IMMUNODEFICIENCY VIRUS 1; HOMO SAPIENS,homo sapiens,homo sapiens,human immunodeficiency virus 1,"Jette, C.A., Bjorkman, P.J.",3.9,ELECTRON MICROSCOPY,NaN,NaN,False,True,IGHV1,IGKV3,Kappa,None,None,None,None,None,1


In [9]:
# don't!
# capitalise Hchain and Lchain
# summary_file["Hchain"] = summary_file["Hchain"].str.capitalize()
# summary_file["Lchain"] = summary_file["Lchain"].str.capitalize()
# summary_file["antigen_chain"] = summary_file["antigen_chain"].str.upper()
# summary_file

In [10]:
data = summary_file[["pdb", "Hchain", "Lchain", "antigen_chain"]]
data.columns = ["pdb", "Hchain", "Lchain", "Achain"]
data.reset_index(drop=True, inplace=True)
data

,pdb,Hchain,Lchain,Achain
0,7t17,J,K,C
1,7t17,F,G,A
2,7jmo,H,L,A
3,4o51,B,A,N
4,7orb,E,F,X
...,...,...,...,...
6534,7tas,H,L,E
6535,7wk0,A,B,C
6536,6ejm,H,h,B
6537,7lo6,J,I,C


In [11]:
num_chains = data.iloc[:, 3][data.iloc[:, 3].str.contains(" | ")].map(lambda x: len(x.split(" | ")))
num_chains.value_counts()

2    578
3     89
4      7
5      1
Name: Achain, dtype: int64

In [12]:
data[data.iloc[:, 3].str.len()>9]

,pdb,Hchain,Lchain,Achain
216,5cws,H,G,K | I | J | L
485,5y9c,H,L,D | A | C | E
2155,6e0c,M,m,C | A | B | D
2582,6lht,H,L,C | A | B | E | D
4857,6e0p,M,m,C | A | B | D
4993,7ssv,H,L,B | A | C | D
5281,5cws,B,A,E | C | D | F
6230,6e0p,N,n,G | E | F | H


In [13]:
data["Hseq"] = data["Hchain"]
data["Lseq"] = data["Lchain"]
data["Aseq"] = data["Achain"]
data["L1"] = data["Lseq"]
data["L2"] = data["Lseq"]
data["L3"] = data["Lseq"]
data["H1"] = data["Hseq"]
data["H2"] = data["Hseq"]
data["H3"] = data["Hseq"]

data

,pdb,Hchain,Lchain,Achain,Hseq,Lseq,Aseq,L1,L2,L3,H1,H2,H3
0,7t17,J,K,C,J,K,C,K,K,K,J,J,J
1,7t17,F,G,A,F,G,A,G,G,G,F,F,F
2,7jmo,H,L,A,H,L,A,L,L,L,H,H,H
3,4o51,B,A,N,B,A,N,A,A,A,B,B,B
4,7orb,E,F,X,E,F,X,F,F,F,E,E,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6534,7tas,H,L,E,H,L,E,L,L,L,H,H,H
6535,7wk0,A,B,C,A,B,C,B,B,B,A,A,A
6536,6ejm,H,h,B,H,h,B,h,h,h,H,H,H
6537,7lo6,J,I,C,J,I,C,I,I,I,J,J,J


In [14]:
data["intersection"] = data["pdb"].apply(lambda x: 1 if x in cdr_seqs.keys() else 0)
data = data[data["intersection"]==1]
data = data.drop(["intersection"], axis=1)
data.reset_index(drop=True, inplace=True)
data

,pdb,Hchain,Lchain,Achain,Hseq,Lseq,Aseq,L1,L2,L3,H1,H2,H3
0,7t17,J,K,C,J,K,C,K,K,K,J,J,J
1,7t17,F,G,A,F,G,A,G,G,G,F,F,F
2,4o51,B,A,N,B,A,N,A,A,A,B,B,B
3,5w08,K,L,C,K,L,C,L,L,L,K,K,K
4,7lg6,J,M,E | B,J,M,E | B,M,M,M,J,J,J
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5383,6d6u,K,L,D,K,L,D,L,L,L,K,K,K
5384,5yax,B,b,C,B,b,C,b,b,b,B,B,B
5385,6ejm,H,h,B,H,h,B,h,h,h,H,H,H
5386,7lo6,J,I,C,J,I,C,I,I,I,J,J,J


In [15]:
data.drop_duplicates(["pdb"])

,pdb,Hchain,Lchain,Achain,Hseq,Lseq,Aseq,L1,L2,L3,H1,H2,H3
0,7t17,J,K,C,J,K,C,K,K,K,J,J,J
2,4o51,B,A,N,B,A,N,A,A,A,B,B,B
3,5w08,K,L,C,K,L,C,L,L,L,K,K,K
4,7lg6,J,M,E | B,J,M,E | B,M,M,M,J,J,J
5,7t6s,E,e,A | B,E,e,A | B,e,e,e,E,E,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5377,1eo8,H,L,A,H,L,A,L,L,L,H,H,H
5378,4ps4,H,L,A,H,L,A,L,L,L,H,H,H
5381,6t3f,H,L,F,H,L,F,L,L,L,H,H,H
5382,4od2,B,A,S,B,A,S,A,A,A,B,B,B


In [16]:
for i in range(data.shape[0]):
    if data.iloc[i,0]=="7t6s":
        print(data.iloc[i, :])

pdb        7t6s
Hchain        E
Lchain        e
Achain    A | B
Hseq          E
Lseq          e
Aseq      A | B
L1            e
L2            e
L3            e
H1            E
H2            E
H3            E
Name: 5, dtype: object


In [17]:
data

,pdb,Hchain,Lchain,Achain,Hseq,Lseq,Aseq,L1,L2,L3,H1,H2,H3
0,7t17,J,K,C,J,K,C,K,K,K,J,J,J
1,7t17,F,G,A,F,G,A,G,G,G,F,F,F
2,4o51,B,A,N,B,A,N,A,A,A,B,B,B
3,5w08,K,L,C,K,L,C,L,L,L,K,K,K
4,7lg6,J,M,E | B,J,M,E | B,M,M,M,J,J,J
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5383,6d6u,K,L,D,K,L,D,L,L,L,K,K,K
5384,5yax,B,b,C,B,b,C,b,b,b,B,B,B
5385,6ejm,H,h,B,H,h,B,h,h,h,H,H,H
5386,7lo6,J,I,C,J,I,C,I,I,I,J,J,J


In [28]:
data_list = []
pdb_structs = glob.glob("../../MSAI_Project/SAbDab_20221124/all_structures/imgt/*.pdb")
pdb_structs = set([idx[-8:-4] for idx in pdb_structs])


for i in tqdm(range(data.shape[0])):
    tmp = {}
    tmp["pdb"] = data.iloc[i, 0]
    tmp["Hchain"] = data.iloc[i, 1]
    tmp["Lchain"] = data.iloc[i, 2]
    tmp["Achain"] = data.iloc[i, 3]
#     if tmp["pdb"] != "7v05":
#         continue
        
    if tmp["pdb"] not in pdb_structs:
        continue

    pdb_path = "../../MSAI_Project/SAbDab_20221124/all_structures/imgt/{}.pdb".format(tmp["pdb"])
    
    p = PDBParser()
    structure = p.get_structure('input', pdb_path)
    all_chains = [c.get_id() for c in structure[0].get_list()]
    
    
    # Chain upper/lower issue
    # In summary.csv, the upper/lower case is not accurately corresponding to pdb files
    # Must manually check
    
    # Hchain
    if tmp["Hchain"] in all_chains:
        pass
    else:
        tmp["Hchain"] = tmp["Hchain"].upper() if tmp["Hchain"].upper() in all_chains else tmp["Hchain"].lower()
    # Lchain
    if tmp["Lchain"] in all_chains:
        pass
    else:
        tmp["Lchain"] = tmp["Lchain"].upper() if tmp["Lchain"].upper() in all_chains else tmp["Lchain"].lower()
    # Achains
    tmp["Achain"] = tmp["Achain"].split(" | ")
    for idx in range(len(tmp["Achain"])):
        if tmp["Achain"][idx] in all_chains:
            pass
        else:
            tmp["Achain"][idx] = tmp["Achain"][idx].upper() if tmp["Achain"][idx].upper() in all_chains else tmp["Achain"][idx].lower()
            
    tmp["Hseq"] = get_residue_seqs(pdb_path, [tmp["Hchain"]], all_chains)
    tmp["Lseq"] = get_residue_seqs(pdb_path, [tmp["Lchain"]], all_chains)
    tmp["Aseq"] = get_residue_seqs(pdb_path, tmp["Achain"], all_chains)

    tmp["L1"] = cdr_seqs[data.iloc[i, 0]][tmp["Hchain"]+tmp["Lchain"]]["L1"]
    tmp["L2"] = cdr_seqs[data.iloc[i, 0]][tmp["Hchain"]+tmp["Lchain"]]["L2"]
    tmp["L3"] = cdr_seqs[data.iloc[i, 0]][tmp["Hchain"]+tmp["Lchain"]]["L3"]
    tmp["H1"] = cdr_seqs[data.iloc[i, 0]][tmp["Hchain"]+tmp["Lchain"]]["H1"]
    tmp["H2"] = cdr_seqs[data.iloc[i, 0]][tmp["Hchain"]+tmp["Lchain"]]["H2"]
    tmp["H3"] = cdr_seqs[data.iloc[i, 0]][tmp["Hchain"]+tmp["Lchain"]]["H3"]
    
    data_list.append(tmp)

#     break

len(data_list)

100%|████████████████████████████████████████████████████████████████████████████| 5388/5388 [1:29:41<00:00,  1.00it/s]


5386

In [20]:
data.iloc[5, :]

pdb        7t6s
Hchain        E
Lchain        e
Achain    A | B
Hseq          E
Lseq          e
Aseq      A | B
L1            e
L2            e
L3            e
H1            E
H2            E
H3            E
Name: 5, dtype: object

In [21]:
data_list[5]

{'pdb': '7t6s',
 'Hchain': 'E',
 'Lchain': 'E',
 'Achain': ['A', 'B'],
 'Hseq': {'E': [{'name': 'VAL',
    'abbr': 'V',
    'pos': array([[142.528, 128.255, 127.442],
           [141.211, 128.935, 127.383],
           [141.445, 130.402, 127.674],
           [142.041, 130.723, 128.708]], dtype=float32)},
   {'name': 'GLN',
    'abbr': 'Q',
    'pos': array([[141.013, 131.257, 126.76 ],
           [141.135, 132.711, 126.953],
           [139.844, 133.387, 126.525],
           [139.34 , 133.082, 125.439]], dtype=float32)},
   {'name': 'LEU',
    'abbr': 'L',
    'pos': array([[139.322, 134.258, 127.382],
           [138.131, 135.056, 127.03 ],
           [138.554, 136.513, 127.119],
           [138.871, 136.987, 128.23 ]], dtype=float32)},
   {'name': 'VAL',
    'abbr': 'V',
    'pos': array([[138.584, 137.188, 125.985],
           [139.051, 138.597, 125.947],
           [137.89 , 139.477, 125.519],
           [137.367, 139.3  , 124.425]], dtype=float32)},
   {'name': 'GLU',
    'abbr': '

In [22]:
data_list[5].keys()

dict_keys(['pdb', 'Hchain', 'Lchain', 'Achain', 'Hseq', 'Lseq', 'Aseq', 'L1', 'L2', 'L3', 'H1', 'H2', 'H3'])

In [23]:
data_list[5]["Hchain"], data_list[5]["Lchain"], data_list[5]["Achain"]

('E', 'E', ['A', 'B'])

In [24]:
"".join([res["abbr"] for res in data_list[5]["Hseq"]["E"]]), len("".join([res["abbr"] for res in data_list[5]["Hseq"]["E"]]))

('VQLVESGGGLVQPGGSRKLSCSASGFAFSSFGMHWVRQAPEKGLEWVAYISSGSGTIYYADTVKGRFTISRDDPKNTLFLQMTSLRSEDTAMYYCVRSIYYYGSSPFDFWGQGTTLTVSSDIVMTQATSSVPVTPGESVSISCRSSKSLLHSNGNTYLYWFLQRPGQSPQLLIYRMSNLASGVPERFSGSGSGTAFTLTISRLEAEDVGVYYCMQHLEYPLTFGAGTKLEL',
 231)

In [25]:
data_list[5]["H1"], data_list[5]["H2"], data_list[5]["H3"]

('GFAFSSFG', 'ISSGSGTI', 'VRSIYYYGSSPFDF')

#### save

In [35]:
with open("./data/data_list.pkl", "wb") as f:
    pickle.dump(data_list, f)
f.close()

In [37]:
# with open("./data/sequence_pairs.json", "w") as f:
#     f.write(json.dumps(data_list))
# f.close()

In [89]:
data_list[i]["pdb"] cdr

'7t17'

In [118]:
cdr_error_cnt = 0

for i in tqdm(range(len(data_list))):
    Hchain = data_list[i]["Hchain"]
    Lchain = data_list[i]["Lchain"]
    Achain = data_list[i]["Achain"]
    
    Hseq = "".join([res["abbr"] for res in data_list[i]["Hseq"][Hchain]])
    Lseq = "".join([res["abbr"] for res in data_list[i]["Lseq"][Lchain]])
    Aseq = ""
    for Achain_k in Achain:
        Aseq = Aseq + "/" + "".join([res["abbr"] for res in data_list[i]["Aseq"][Achain_k]])
        
        
    H1, H2, H3 = data_list[i]["H1"], data_list[i]["H2"], data_list[i]["H3"]
    L1, L2, L3 = data_list[i]["L1"], data_list[i]["L2"], data_list[i]["L3"]

    is_ok = True
    cdr_span = {}
    for cdr in ["H1", "H2", "H3", "L1", "L2", "L3"]:
        if cdr[0]=="H":
            start = Hseq.find(data_list[i][cdr])
            end = start + len(data_list[i][cdr])
        if cdr[0]=="L":
            start = Lseq.find(data_list[i][cdr])
            end = start + len(data_list[i][cdr])
            
        if start != -1:
            cdr_span[cdr] = (start, end)
        else:
            print("idx={} pdb:{} {} doesn't exist".format(i, data_list[i]["pdb"], cdr))
            is_ok = False
            cdr_error_cnt += 1
            break
    if is_ok==False:
        continue    
    
#     data_list[i]["L1"]
    
#     break

 60%|████████████████████████████████████████████▋                              | 3209/5386 [00:00<00:00, 16179.26it/s]

idx=31 pdb:7um3 H1 doesn't exist
idx=45 pdb:4ydv L1 doesn't exist
idx=59 pdb:5dup H2 doesn't exist
idx=65 pdb:3mlt L3 doesn't exist
idx=114 pdb:8d5c H1 doesn't exist
idx=134 pdb:5a2i H1 doesn't exist
idx=182 pdb:4f15 H3 doesn't exist
idx=194 pdb:6b3m H3 doesn't exist
idx=198 pdb:5csz H3 doesn't exist
idx=215 pdb:4m1c L1 doesn't exist
idx=230 pdb:3csy H3 doesn't exist
idx=237 pdb:6mvl L1 doesn't exist
idx=287 pdb:7fei H3 doesn't exist
idx=308 pdb:6wty H1 doesn't exist
idx=331 pdb:4m8q H3 doesn't exist
idx=358 pdb:2znx H3 doesn't exist
idx=368 pdb:6e1k L1 doesn't exist
idx=442 pdb:6wty H1 doesn't exist
idx=447 pdb:6b3m H3 doesn't exist
idx=488 pdb:7t9b H3 doesn't exist
idx=495 pdb:5tq0 L2 doesn't exist
idx=510 pdb:5ies L1 doesn't exist
idx=522 pdb:4tul H1 doesn't exist
idx=525 pdb:6p8n L1 doesn't exist
idx=529 pdb:3qnz H3 doesn't exist
idx=570 pdb:7k39 L1 doesn't exist
idx=601 pdb:4r4n H1 doesn't exist
idx=608 pdb:7jtr L1 doesn't exist
idx=615 pdb:7x1u L2 doesn't exist
idx=629 pdb:6kvf H

100%|███████████████████████████████████████████████████████████████████████████| 5386/5386 [00:00<00:00, 15795.21it/s]

idx=4529 pdb:4ydv L1 doesn't exist
idx=4533 pdb:7k37 L1 doesn't exist
idx=4561 pdb:6p8m L1 doesn't exist
idx=4599 pdb:6u0n H3 doesn't exist
idx=4608 pdb:3lh2 H3 doesn't exist
idx=4609 pdb:6glx H1 doesn't exist
idx=4613 pdb:4m8q H3 doesn't exist
idx=4617 pdb:6nf5 L3 doesn't exist
idx=4646 pdb:6j14 H1 doesn't exist
idx=4649 pdb:6xrt H3 doesn't exist
idx=4677 pdb:7e9q H3 doesn't exist
idx=4687 pdb:6woq H1 doesn't exist
idx=4700 pdb:5wob L1 doesn't exist
idx=4791 pdb:7mk6 L2 doesn't exist
idx=4816 pdb:6nf5 L3 doesn't exist
idx=4827 pdb:4y5y H1 doesn't exist
idx=4843 pdb:6u0n H3 doesn't exist
idx=4854 pdb:7ekb H3 doesn't exist
idx=4923 pdb:7x1t L3 doesn't exist
idx=4927 pdb:7e9o H3 doesn't exist
idx=4928 pdb:3mlt L3 doesn't exist
idx=4934 pdb:7wkd L2 doesn't exist
idx=4944 pdb:5wob L1 doesn't exist
idx=5032 pdb:6kdi L1 doesn't exist
idx=5042 pdb:6wty H1 doesn't exist
idx=5090 pdb:7jtr L1 doesn't exist
idx=5145 pdb:3gbm L3 doesn't exist
idx=5157 pdb:6kn9 L1 doesn't exist
idx=5183 pdb:3tt3 H3

In [124]:
hseq = "".join([res["abbr"] for res in data_list[31]["Hseq"][data_list[31]["Hchain"]]])
hseq

'VQLQQWGAGLLKPSETLSLTCAVYGFSDYYWNWIRQPPGKGLEWIGEINHNGNTNSNPSLKSRVTLSLDTSKNQFSLKLRSVTAADTAVYYCAFGYSDYEYNWFDPWGQGTLVTVSSASTKGPSVFPLAPSSGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKRVEPKS*************************************************'

In [127]:
data_list[31]["H1"], data_list[31]["H2"], data_list[31]["H3"]

('GGSFSDYY', 'INHNGNT', 'AFGYSDYEYNWFDP')

In [117]:
cdr_error_cnt

254

In [100]:
Hseq

'QVQLQESGPGLVKPSQTLSLTCAVSGGSISSGDSYWSWIRQHPGKGLEWIGSIYYSGSTYYNPSLKSRVTIPIDTSKNQFSLKLSSVTAADTAVYYCARHVGDLRVNDAFDIWGQGTMVTVSS'

In [101]:
data_list[i]["H1"], data_list[i]["H2"], data_list[i]["H3"]

('GGSISSGDSY', 'IYYSGST', 'ARHVGDLRVNDAFDI')

In [102]:
H1_span, H2_span, H3_span

((25, 35), (52, 59), (97, 112))

In [103]:
Hseq[H1_span[0]:H1_span[1]], Hseq[H2_span[0]:H2_span[1]], Hseq[H3_span[0]:H3_span[1]]

('GGSISSGDSY', 'IYYSGST', 'ARHVGDLRVNDAFDI')

In [104]:
Lseq

'QSVLTQPPSVSAAPGQKVTISCSGSSSNIGNNFVSWYQRLPGTPPKLLIYDSDKRPSGIPDRFSGSKSGTSATLGITGLQTGDEGDYYCGTWDRSLSVVVFGGGTKLTVL'

In [107]:
data_list[i]["L1"], data_list[i]["L2"], data_list[i]["L3"]

('SSNIGNNF', 'DSD', 'GTWDRSLSVVV')

In [106]:
cdr_span

{'H1': (25, 35),
 'H2': (52, 59),
 'H3': (97, 112),
 'L1': (25, 33),
 'L2': (50, 53),
 'L3': (89, 100)}

In [112]:
data_list[i]["Lseq"].keys()

dict_keys(['K'])

In [113]:
data_list[i]["Lseq"]["K"][50:53]

[{'name': 'ASP',
  'abbr': 'D',
  'pos': array([[ 37.754, -32.75 , 222.722],
         [ 36.301, -32.899, 222.789],
         [ 35.885, -34.3  , 223.229],
         [ 34.715, -34.677, 223.104]], dtype=float32)},
 {'name': 'SER',
  'abbr': 'S',
  'pos': array([[ 36.842, -35.082, 223.74 ],
         [ 36.601, -36.439, 224.22 ],
         [ 36.477, -37.496, 223.124],
         [ 36.322, -38.682, 223.433]], dtype=float32)},
 {'name': 'ASP',
  'abbr': 'D',
  'pos': array([[ 36.529, -37.099, 221.851],
         [ 36.441, -38.069, 220.767],
         [ 37.496, -37.849, 219.688],
         [ 37.985, -38.811, 219.088]], dtype=float32)}]

# remove repeated or similar (>95%) samples
- criterion:
- TODO

In [39]:
data_list = pickle.load(open("./data/data_list.pkl", "rb"))
len(data_list)

5386

In [ ]:
i = 0
data_list[i][Lchain]

In [51]:
data_list[0]["Lseq"][data_list[0]["Lchain"]]

[{'name': 'GLN',
  'abbr': 'Q',
  'pos': array([[ 37.846, -27.322, 244.186],
         [ 36.957, -28.308, 243.585],
         [ 37.441, -28.704, 242.194],
         [ 36.769, -29.451, 241.482]], dtype=float32)},
 {'name': 'SER',
  'abbr': 'S',
  'pos': array([[ 38.613, -28.197, 241.812],
         [ 39.191, -28.502, 240.508],
         [ 39.999, -29.793, 240.556],
         [ 41.229, -29.767, 240.446]], dtype=float32)},
 {'name': 'VAL',
  'abbr': 'V',
  'pos': array([[ 39.318, -30.924, 240.72 ],
         [ 39.96 , -32.231, 240.788],
         [ 39.226, -33.173, 239.845],
         [ 37.992, -33.25 , 239.873]], dtype=float32)},
 {'name': 'LEU',
  'abbr': 'L',
  'pos': array([[ 39.981, -33.882, 239.012],
         [ 39.441, -34.906, 238.128],
         [ 39.932, -36.258, 238.628],
         [ 41.141, -36.512, 238.651]], dtype=float32)},
 {'name': 'THR',
  'abbr': 'T',
  'pos': array([[ 38.999, -37.117, 239.026],
         [ 39.357, -38.408, 239.595],
         [ 39.738, -39.386, 238.492],
         [ 

In [45]:
len(data_list[0]["L1"]), len(data_list[0]["Lseq"][data_list[0]["Lchain"]])

(8, 110)

In [38]:
# with open("./data/sequence_pairs.json", "rb") as f:
#     cdr_seqs = json.loads(f.read())
# f.close()

In [31]:
# type(cdr_seqs), len(cdr_seqs)

(list, 5388)